# RANDOM FOREST review

In [1]:
import pandas as pd
import numpy as np

In [2]:
categorical=pd.read_csv('categorical.csv')

In [3]:
numerical=pd.read_csv('numerical.csv')

In [4]:
target=pd.read_csv('target.csv')

In [5]:
categorical.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3


In [6]:
numerical.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


In [7]:
target.head()

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [8]:
categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STATE         95412 non-null  object
 1   CLUSTER       95412 non-null  int64 
 2   HOMEOWNR      95412 non-null  object
 3   GENDER        95412 non-null  object
 4   DATASRCE      95412 non-null  int64 
 5   RFA_2R        95412 non-null  object
 6   RFA_2A        95412 non-null  object
 7   GEOCODE2      95412 non-null  object
 8   DOMAIN_A      95412 non-null  object
 9   DOMAIN_B      95412 non-null  int64 
 10  ODATEW_YR     95412 non-null  int64 
 11  ODATEW_MM     95412 non-null  int64 
 12  DOB_YR        95412 non-null  int64 
 13  DOB_MM        95412 non-null  int64 
 14  MINRDATE_YR   95412 non-null  int64 
 15  MINRDATE_MM   95412 non-null  int64 
 16  MAXRDATE_YR   95412 non-null  int64 
 17  MAXRDATE_MM   95412 non-null  int64 
 18  LASTDATE_YR   95412 non-null  int64 
 19  LAST

In [9]:
numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 315 entries, TCODE to CLUSTER2
dtypes: float64(9), int64(306)
memory usage: 229.3 MB


In [10]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TARGET_B  95412 non-null  int64  
 1   TARGET_D  95412 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.5 MB


In [11]:
data=pd.concat([numerical,categorical,target], axis=1)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 339 entries, TCODE to TARGET_D
dtypes: float64(10), int64(322), object(7)
memory usage: 246.8+ MB


In [13]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

## imbalance management - downsampling

In [14]:
category_0=data[data['TARGET_B']==0].sample(len(data[data['TARGET_B']==1]))

In [15]:
category_1=data[data['TARGET_B']==1]

In [16]:
data=pd.concat([category_0, category_1], axis=0)

In [17]:
data.shape

(9686, 339)

In [18]:
data=data.reset_index(drop=True)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9686 entries, 0 to 9685
Columns: 339 entries, TCODE to TARGET_D
dtypes: float64(10), int64(322), object(7)
memory usage: 25.1+ MB


### set dependent variable


In [20]:
y=data['TARGET_B']
X=data.drop(['TARGET_B'], axis=1)

In [21]:
X_num=X.select_dtypes(include=np.number)
X_cat=X.select_dtypes(object)

In [22]:
X_cat.shape

(9686, 7)

In [23]:
from sklearn.preprocessing import OneHotEncoder

In [24]:
encoder=OneHotEncoder(drop='first').fit(X_cat)

In [25]:
X_cat_en=encoder.transform(X_cat).toarray() # lost categorical feature names

In [26]:
X_cat_en_cols=encoder.get_feature_names_out(X_cat.columns) # restore feature names / older version get_feature_names

In [27]:
X_cat_en_cols

array(['STATE_FL', 'STATE_GA', 'STATE_IL', 'STATE_IN', 'STATE_MI',
       'STATE_MO', 'STATE_NC', 'STATE_TX', 'STATE_WA', 'STATE_WI',
       'STATE_other', 'HOMEOWNR_U', 'GENDER_M', 'GENDER_other',
       'RFA_2A_E', 'RFA_2A_F', 'RFA_2A_G', 'GEOCODE2_B', 'GEOCODE2_C',
       'GEOCODE2_D', 'DOMAIN_A_R', 'DOMAIN_A_S', 'DOMAIN_A_T',
       'DOMAIN_A_U'], dtype=object)

In [28]:
X_cat_en

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
X_cat_en_df=pd.DataFrame(X_cat_en) # lost categorical feature names

In [30]:
X_cat_en_df=pd.DataFrame(X_cat_en, columns=X_cat_en_cols) # restore categorical names

In [31]:
X_cat_en_df

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [32]:
X_full=pd.concat([X_cat_en_df,X_num], axis=1) # lost numerical feature names / retained feature names

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train,X_test,y_train,y_test=train_test_split(X_full,y,test_size=0.3,random_state=40)

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
clf=RandomForestClassifier(n_estimators=200, max_depth=2, random_state=40)
clf.fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.9036476256022024


## with ensemble you should use more than one scoring metric

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
cross_val_scores=cross_val_score(clf, X_train, y_train)

In [39]:
cross_val_scores # the closer the number are toghter more accurated it is. Also there are more parameters qe have not set up. Cross Validation always split in 5 pieces

array([0.90929204, 0.91961652, 0.89528024, 0.94542773, 0.91961652])

## Feature Selection / scoring with Random Forest

In [41]:
feature_importances=clf.feature_importances_

In [42]:
feature_names=X_full.columns

In [43]:
forest_importances= pd.Series(feature_importances, index=feature_names)

In [44]:
forest_importances.sort_values(ascending=False)

TARGET_D    0.110456
AVGGIFT     0.066611
LASTGIFT    0.058338
CARDGIFT    0.055233
RFA_2F      0.054667
              ...   
TPE11       0.000000
TPE13       0.000000
LFC1        0.000000
LFC2        0.000000
STATE_FL    0.000000
Length: 355, dtype: float64

In [45]:
with pd.option_context('display.max_rows', None):
    print(forest_importances.sort_values(ascending=False))

TARGET_D        0.110456
AVGGIFT         0.066611
LASTGIFT        0.058338
CARDGIFT        0.055233
RFA_2F          0.054667
MINRAMNT        0.046625
NGIFTALL        0.038932
MAXRAMNT        0.032393
LASTDATE_YR     0.024133
RAMNTALL        0.021523
IC4             0.019449
CARDPROM        0.019202
ODATEW_YR       0.016864
MAXRDATE_YR     0.016510
RFA_2A_E        0.015008
NUMPRM12        0.014251
FIRSTDATE_YR    0.012718
HV1             0.011309
CARDPM12        0.011001
IC3             0.010501
NUMPROM         0.009397
HVP5            0.008870
DOMAIN_B        0.008805
IC5             0.008518
EC2             0.008509
IC6             0.008449
ETH1            0.008188
HV2             0.008126
DMA             0.007737
IC2             0.007626
RFA_2A_F        0.006433
IC19            0.005941
MHUC1           0.005763
EC5             0.005754
TIMELAG         0.005728
HHAS3           0.005628
IC21            0.005625
LASTDATE_MM     0.005452
DOB_YR          0.005395
RFA_2A_G        0.005246
